In [1]:
!pip install --upgrade datasets fsspec transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.0/201.0 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 19.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pya

In [2]:
# import datasets, tokenizer, model, trainer, training arguments

from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments



In [3]:
# -- 1) load dataset
dataset = load_dataset("imdb")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [5]:
train_dataset = dataset["train"].select(range(1000)) # get 1000 for train
test_dataset = dataset["test"].select(range(200)) # get 200 for test

In [6]:
# -- 2) tokanize

# initialize tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [7]:
# tokenize function - tokenization, padding, truncation adn max length

def tokenize_fn(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)


In [8]:
# preprocess function - apply tokenization, remove column text(save memory), format to torch

def preprocess(ds):
    ds = ds.map(tokenize_fn, batched=True, remove_columns=["text"])
    ds = ds.rename_column("label", "labels")
    ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
    return ds


In [9]:
train_dataset = preprocess(train_dataset) # preprocess the train dataset

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [10]:
test_dataset = preprocess(test_dataset) # preprocess the test dataset

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [11]:
# -- 3) model

# initialize model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# #Classifier head trainable rahega by default

# for param in model.bert.parameters():
#     param.requires_grad = False  # Freeze BERT encoder

# #Unfreeze last 2 encoder layers

# for layer in model.bert.encoder.layer[-2:]:
#     for param in layer.parameters():
#         param.requires_grad = True

In [13]:
# -- 4) training arguments

In [14]:
# training arguments

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./bert-finetuned-imdb",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    logging_dir="./logs",
    learning_rate=2e-5,
    weight_decay=0.01,
    report_to="none"
)

In [15]:
# -- 5) trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)


In [16]:
# -- 6) train
trainer.train()

Step,Training Loss


TrainOutput(global_step=125, training_loss=0.03880718231201172, metrics={'train_runtime': 52.3452, 'train_samples_per_second': 19.104, 'train_steps_per_second': 2.388, 'total_flos': 131555527680000.0, 'train_loss': 0.03880718231201172, 'epoch': 1.0})

In [17]:
# !tensorboard --logdir=./logs

In [18]:
trainer.save_model("./bert-finetuned-imdb")

In [19]:
tokenizer.save_pretrained("./bert-finetuned-imdb")

('./bert-finetuned-imdb/tokenizer_config.json',
 './bert-finetuned-imdb/special_tokens_map.json',
 './bert-finetuned-imdb/vocab.txt',
 './bert-finetuned-imdb/added_tokens.json')

In [20]:
# -- 7) evaluate
metrics = trainer.evaluate()

In [21]:
print(metrics)

{'eval_loss': 0.001158248633146286, 'eval_runtime': 2.7609, 'eval_samples_per_second': 72.441, 'eval_steps_per_second': 9.055, 'epoch': 1.0}


In [22]:
# -- 8) prediction

In [23]:
tokenizer = BertTokenizer.from_pretrained("/content/bert-finetuned-imdb")

In [24]:
model = BertForSequenceClassification.from_pretrained("/content/bert-finetuned-imdb")

In [26]:
from transformers import pipeline
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [27]:
# Predict
text = "This movie was amazing and I loved the acting!"
result = classifier(text)

In [28]:
print(result)  # Example: [{'label': 'POSITIVE', 'score': 0.98}]

[{'label': 'LABEL_0', 'score': 0.9956529140472412}]
